In [4]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re

In [5]:
#data loader class
class DataLoader:
  def __init__(self,dir_name,file_name):
    self.dir_name=dir_name
    self.file_name = file_name
    
 
  def read_csv(self):
    os.chdir(self.dir_name)
    tweets_df=pd.read_csv(self.file_name)
    return tweets_df
  
    

In [6]:
DataLoader_obj= DataLoader('../Notebooks','cleaned_fintech_data.csv')

In [7]:
tweets_df=DataLoader_obj.read_csv()
tweets_df.dropna()


,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,...,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp


In [13]:
len(tweets_df)

5621

In [8]:
class PrepareData:
  def __init__(self,df):
    self.df=df
    
  def preprocess_data(self):
    tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
    tweets_df['clean_text']=tweets_df['clean_text'].astype(str)
    tweets_df['clean_text'] = tweets_df['clean_text'].apply(lambda x: x.lower())
    tweets_df['clean_text']= tweets_df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    #Converting tweets to list of words For feature engineering
    sentence_list = [tweet for tweet in tweets_df['clean_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word 
    word_to_id = corpora.Dictionary(word_list)
    corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]



    
    return word_list, word_to_id, corpus_1


In [9]:
PrepareData_obj=PrepareData(tweets_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [10]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(29, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 2), (56, 2), (57, 1)], [(58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(29, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1),

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('carbon', 0.034708485),
   ('climate', 0.024370827),
   ('gt', 0.024116777),
   ('bp', 0.014439733),
   ('change', 0.012907533),
   ('future', 0.009547638),
   ('land', 0.009546835),
   ('large', 0.008649601),
   ('intense', 0.008456625),
   ('focus', 0.007898001)]),
 (1,
  [('money', 0.024699867),
   ('account', 0.017653052),
   ('like', 0.014913489),
   ('ur', 0.014439795),
   ('june', 0.014196563),
   ('follow', 0.013304843),
   ('giveaway', 0.013050717),
   ('giving', 0.013021249),
   ('would', 0.012878241),
   ('rt', 0.0125428075)]),
 (2,
  [('carbon', 0.024277104),
   ('nt', 0.016590921),
   ('i', 0.01616982),
   ('meat', 0.014486317),
   ('uk', 0.013002311),
   ('footprint', 0.01136055),
   ('s', 0.009493683),
   ('go', 0.009141456),
   ('low', 0.008937163),
   ('great', 0.008675809)]),
 (3,
  [('carbon', 0.043205258),
   ('tax', 0.026049877),
   ('protecting', 0.025413387),
   ('rights', 0.02533519),
   ('amp', 0.024646819),
   ('oil', 0.021254148),
   ('emissions', 0.

In [13]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -6.7493284853454405

 Ldamodel Coherence Score/Accuracy on Tweets:  0.6171123204998623


In [14]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared 

C:\Users\euelf\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\euelf\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=Tru

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.352920  0.012811       1        1  30.180600
2     -0.169380  0.168976       2        1  21.181278
1     -0.093829 -0.354380       3        1  19.827345
4      0.021438  0.075665       4        1  17.670918
0     -0.111148  0.096928       5        1  11.139859, topic_info=            Term        Freq        Total Category  logprob  loglift
364   government  446.000000   446.000000  Default  30.0000  30.0000
973          tax  622.000000   622.000000  Default  29.0000  29.0000
966   protecting  607.000000   607.000000  Default  28.0000  28.0000
971       rights  605.000000   605.000000  Default  27.0000  27.0000
50     emissions  821.000000   821.000000  Default  26.0000  26.0000
...          ...         ...          ...      ...      ...      ...
263      climate  214.648937   302.794174   Topic5  -3.7144   1.8506
558       future   84.091948   112.584949   Topic5  -4.6515   1.9028
29        carbon  305.699083  2210.988270   Topic5  -3.3608   0.2160
1752       large   76.182380   116.605738   Topic5  -4.7502   1.7690
37          land   84.084877   393.107351   Topic5  -4.6515   0.6524

[223 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
58        3  0.998097    account
1186      3  0.998140       accs
4157      5  0.987747     actual
951       1  0.996062    against
4319      5  0.982381  americans
...     ...       ...        ...
300       3  0.960894      would
1208      3  0.998140       xtra
277       2  0.988852       year
950       4  0.980470        you
373       4  0.997065       zero

[222 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 5, 1])